In [ ]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

# fine tune model

In [ ]:
#yolo settings datasets_dir='/Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/Segmentation/tree-disk-segmentation.v3i.yolov12'

In [ ]:
base_model_path = "../models/yolo11s-seg.pt"
dataset_path = "/Volumes/Tony SSD/Projekte/Studienarbeit/Datasets/Segmentation/tree-disk-segmentation.v3i.yolov12/data.yaml"

In [ ]:
# check if the dataset exists
if not os.path.exists(dataset_path):
    print("Dataset not found")
    exit()

# check if model exists
if not os.path.exists(base_model_path):
    print("Model not found")
    exit()

In [ ]:
# Load a model
base_model = YOLO(base_model_path)

# Train the model
train_results = base_model.train(
    data=dataset_path,  # path to dataset YAML
    epochs=100,  # number of training epochs
    imgsz=640,  # training image size
    device="cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    batch=32, # batch size
)

In [ ]:
# Evaluate model performance on the validation set
metrics = base_model.val()

# results

## Plot segmented masks

In [ ]:
trained_model_path = "../models/yolo11s-seg-tree.pt"

In [ ]:
# check if the model exists
if not os.path.exists(trained_model_path):
    print("Model not found")
    exit()

In [ ]:
trained_model = YOLO(trained_model_path)

In [ ]:
results = trained_model("../input/tree-disk4.jpg")

## Plot segmentation mask with original colors

In [ ]:
for result in results:
    height, width = result.orig_img.shape[:2]
    background = np.ones((height, width, 3), dtype=np.uint8) * 255
    masks = result.masks.xy
    orig_img = result.orig_img

    for mask in masks:
        mask = mask.astype(int)
        mask_img = np.zeros_like(orig_img)

        cv2.fillPoly(mask_img, [mask], (255, 255, 255))
        masked_object = cv2.bitwise_and(orig_img, mask_img)
        background[mask_img == 255] = masked_object[mask_img == 255]

    background_rgb = cv2.cvtColor(background, cv2.COLOR_BGR2RGB)

    plt.imshow(background_rgb)
    plt.title('Segmented objects')
    plt.axis('off')
    plt.show()

    cv2.imwrite('segmented_objects.jpg', background)

# generate dataset for pith detection

In [ ]:
dataset_path = "/Users/tonymeissner/Downloads/bilder"
output_folder = "output"
model_path = "/Users/tonymeissner/source/tree-disk-analyzer/tree-disk-segmentation/models/yolo11s-seg-tree.pt"

In [ ]:
# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function to check if file is a valid image
def is_valid_image(filepath):
    # Skip macOS resource fork files and hidden files
    if os.path.basename(filepath).startswith('._') or os.path.basename(filepath).startswith('.'):
        return False
    
    # Check file extension
    if not filepath.lower().endswith('.jpg'):
        return False
    
    # Verify file exists
    if not os.path.exists(filepath):
        return False
        
    return True

# Find all valid image files
image_paths = []
for file in os.listdir(dataset_path):
    full_path = os.path.join(dataset_path, file)
    if os.path.isfile(full_path) and is_valid_image(full_path):
        image_paths.append(full_path)

print(f"Found {len(image_paths)} valid images")

In [ ]:
import treedisksegmentation
import logging
import gc

def process_image(image_path, output_folder, model_path):
    """Process a single image with YOLO segmentation."""
    try:
        output_path = os.path.join(output_folder, os.path.basename(image_path))

        if os.path.exists(output_path):
            print(f"Skipping existing output: {image_path}")
            return True

        treedisksegmentation.configure(
            model_path=model_path,
            input_image=image_path,
        )

        result_image, masks = treedisksegmentation.run()

        if masks is None:
            print(f"No masks detected for image: {image_path}")
            return False

        result_image = cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, result_image)
        print(f"Successfully processed: {image_path}")
        return True

    except Exception as e:
        logging.error(f"Error processing {image_path}: {str(e)}", exc_info=True)
        return False

In [ ]:
for image_path in image_paths:
    process_image(image_path, output_folder, model_path)
    print(f"{image_paths.index(image_path)} / {len(image_paths)} images processed")
    gc.collect()